<a href="https://colab.research.google.com/github/rrankawat/stm32/blob/main/CIFAR10_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Model Training

In [1]:
!pip -q install onnx onnxruntime onnxscript onnxruntime-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 128.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00


In [2]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import onnx
import onnxruntime as ort

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
# Data Loaders
batch_size = 64

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

100%|██████████| 170M/170M [00:04<00:00, 41.7MB/s]


In [4]:
# CNN Model
class CIFARConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.fc1 = nn.Linear(128 * 2 * 2, 256)
        self.fc2 = nn.Linear(256, 10)

        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2) # 32 -> 16

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2) # 16 -> 8

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2) # 8 -> 4

        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2) # 4 -> 2

        x = x.view(x.size(0), -1) # Flatten

        # x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [5]:
# Random Seeds and Model Instance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(41)
model = CIFARConvNet().to(device)

In [6]:
# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Training
def train(model, loader, criterion, optimizer, device):
    model.train() # training mode

    correct = 0
    total = 0
    running_loss = 0

    for X, y in loader:
        X = X.to(device)
        y = y.to(device)

        # Forward
        outputs = model(X)

        # Loss
        loss = criterion(outputs, y)

        # Backward
        optimizer.zero_grad(set_to_none = True)
        loss.backward()
        optimizer.step()

        # Statistics
        batch_size = y.size(0)
        running_loss += loss.item() * batch_size
        preds = outputs.argmax(dim = 1)
        correct += (preds == y).sum().item()
        total += batch_size

    avg_loss = running_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

In [8]:
# Testing
def test(model, loader, criterion):
  model.eval() # testing mode

  correct = 0
  total = 0
  running_loss = 0

  with torch.no_grad():
    for X, y in loader:
      X = X.to(device)
      y = y.to(device)

      # Forward
      outputs = model(X)

      # Loss
      loss = criterion(outputs, y)

      # Statistics
      batch_size = y.size(0)
      running_loss += loss.item() * batch_size
      preds = outputs.argmax(dim = 1)
      correct += (preds == y).sum().item()
      total += batch_size

  avg_loss = running_loss / total
  accuracy = correct / total

  return avg_loss, accuracy

In [9]:
epochs = 5
start_time = time.time()

for epoch in range(1, epochs + 1):
  train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
  test_loss, test_acc = test(model, test_loader, criterion)

  print(
      f"Epoch: {epoch}/{epochs} | "
      f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
      f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%"
  )

print(f"Training Time: {(time.time() - start_time) / 60} minutes!")

Epoch: 1/5 | Train Loss: 1.4171 | Train Acc: 0.48% | Test Loss: 1.1457 | Test Acc: 0.59%
Epoch: 2/5 | Train Loss: 1.0720 | Train Acc: 0.62% | Test Loss: 0.9256 | Test Acc: 0.67%
Epoch: 3/5 | Train Loss: 0.9512 | Train Acc: 0.66% | Test Loss: 0.8259 | Test Acc: 0.71%
Epoch: 4/5 | Train Loss: 0.8761 | Train Acc: 0.69% | Test Loss: 0.8510 | Test Acc: 0.71%
Epoch: 5/5 | Train Loss: 0.8262 | Train Acc: 0.71% | Test Loss: 0.7922 | Test Acc: 0.72%
Training Time: 3.423523783683777 minutes!


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
torch.save(model.state_dict(), "/content/drive/My Drive/Colab Notebooks/stm_cifar10_model.pth")
print("✅ Model saved as stm_cifar10_model.pth")

✅ Model saved as stm_cifar10_model.pth
